In [1]:
from collections import defaultdict
import pandas as pd
import numpy as np
import math
import statistics
import re
import json

In [2]:
business=pd.read_csv("yelp_business.csv")

In [3]:
business= business[business.state == "IL"]
business= business[business.is_open == 1]


In [4]:
def category_splitter(category_string):
    categories = category_string.split(";")   
    return categories
business['categories'] = business['categories'].map(lambda x: category_splitter(x))


In [5]:
def first_list(category_list):
    category_list = category_list[0]
    return category_list
business['categories'] = business['categories'].map(lambda x: first_list(x).lower())
business['categories'] = business['categories'].map(lambda x: re.sub(r'\'', '', x).lower())

business.drop(["business_id","neighborhood","address","review_count","is_open"], axis=1)


,name,city,state,postal_code,latitude,longitude,stars,categories
70,"""Carle Clinic Association""",Urbana,IL,61801,40.116627,-88.214763,2.5,doctors
190,"""Zorba's Restaurant""",Champaign,IL,61820,40.110085,-88.229304,4.0,restaurants
239,"""Dunkin' Donuts""",Champaign,IL,61820,40.110107,-88.229991,4.0,food
362,"""Essential Healing Therapeutic Massage & Bodyw...",Savoy,IL,61874,40.062864,-88.250673,5.0,massage therapy
398,"""The Illini Shop""",Champaign,IL,61820,40.109698,-88.230560,3.0,sporting goods
416,"""K-Bowl""",Champaign,IL,61820,40.109988,-88.240506,3.5,restaurants
477,"""Activities and Recreation Center - ARC""",Champaign,IL,61820,40.100830,-88.235556,4.0,gyms
504,"""Bell Sports""",Rantoul,IL,61866,40.312415,-88.095334,4.5,shopping
556,"""Action Rents""",Rantoul,IL,61866,40.305551,-88.155305,2.5,event planning & services
562,"""Glenn Poor's Audio-Video""",Champaign,IL,61821,40.098235,-88.274313,5.0,shopping


In [6]:
business.to_csv("il_data.csv")

In [7]:
# for typeahead words
raw_unique_cats = list(np.unique(business.categories))

unique_cats = [re.sub(r'\'', '', i).lower() for i in raw_unique_cats]

print(unique_cats)

['accessories', 'active life', 'acupuncture', 'adult entertainment', 'african', 'air duct cleaning', 'airport shuttles', 'american (new)', 'american (traditional)', 'amusement parks', 'apartments', 'appliances', 'appliances & repair', 'arcades', 'art galleries', 'art schools', 'arts & crafts', 'arts & entertainment', 'asian fusion', 'auto detailing', 'auto glass services', 'auto insurance', 'auto parts & supplies', 'auto repair', 'automotive', 'bagels', 'bakeries', 'banks & credit unions', 'barbeque', 'barbers', 'bars', 'basque', 'beauty & spas', 'bed & breakfast', 'beer', 'beer bar', 'bike repair/maintenance', 'bikes', 'body shops', 'books', 'bookstores', 'bowling', 'breakfast & brunch', 'breweries', 'bridal', 'buffets', 'building supplies', 'burgers', 'buses', 'cafes', 'candy stores', 'car dealers', 'car rental', 'car wash', 'carpet cleaning', 'carpeting', 'caterers', 'cheese shops', 'chicken wings', 'childrens clothing', 'chimney sweeps', 'chinese', 'chiropractors', 'chocolatiers & 

# Create JSON of Map Grid of Boxes

### Filter city area out of all businesses

In [8]:
# Champaign Area
min_long = -88.341485
max_long = -88.161925
min_lat = 40.060123
max_lat = 40.169594

city_biz = business[(business.latitude < max_lat) &
                         (business.latitude > min_lat) &
                         (business.longitude < max_long) &
                         (business.longitude > min_long)]

### Compute lat and long increments

In [9]:
# Compute degrees lat per mile (N-S) and degrees long per mile (E-W)
ns_deg_per_mi = 1/69.01
ew_deg_per_mi = 1/(math.cos(
                            statistics.mean([min_lat, max_lat])*(math.pi/180)
                            ) * 69.1710411)

# Compute miles NS and EW for selected city area
miles_ns = (max_lat - min_lat)/ns_deg_per_mi
miles_ew = (max_long - min_long)/ew_deg_per_mi

delta_long = (max_long - min_long)/miles_ew
delta_lat = (max_lat - min_lat)/miles_ns

### Get counts of number of each category in a grid box

In [10]:
def get_category_counts(minimum_lat, minimum_long, d_lat, d_long):
    category_dict = defaultdict(int)
    for column, biz in business.iterrows():
        if ((biz['latitude'] > minimum_lat) and (biz['latitude'] < minimum_lat + d_lat)) and ((biz['longitude'] > minimum_long) and (biz['longitude'] < minimum_long + d_long)):
            category_dict[biz['categories']] += 1
    if not category_dict:
        category_dict['no business found in this area'] += 1
    return category_dict
        

### Create list of grid box dictionaries

In [11]:
boxes = []
 
i = min_lat
while i < max_lat:
    j = min_long
    while j < max_long:
        cat_counts = get_category_counts(i, j, delta_lat, delta_long)
        box = {'min_lat': i,
              'max_lat': i + delta_lat,
              'min_long': j,
              'max_long': j + delta_long,
              'category_counts': cat_counts}
        print(box)
        boxes.append(box)
        j += delta_long
    i += delta_lat
        

{'min_lat': 40.060123, 'max_lat': 40.07461365352847, 'min_long': -88.341485, 'max_long': -88.32258099861983, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 40.060123, 'max_lat': 40.07461365352847, 'min_long': -88.32258099861983, 'max_long': -88.30367699723965, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 40.060123, 'max_lat': 40.07461365352847, 'min_long': -88.30367699723965, 'max_long': -88.28477299585947, 'category_counts': defaultdict(<class 'int'>, {'flowers & gifts': 1, 'restaurants': 1})}
{'min_lat': 40.060123, 'max_lat': 40.07461365352847, 'min_long': -88.28477299585947, 'max_long': -88.26586899447929, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 40.060123, 'max_lat': 40.07461365352847, 'min_long': -88.26586899447929, 'max_long': -88.24696499309911, 'category_counts': defaultdict(<class 'int'>, {'massage therapy': 3, 'shoe s

{'min_lat': 40.08910430705695, 'max_lat': 40.103594960585426, 'min_long': -88.20915699033876, 'max_long': -88.19025298895858, 'category_counts': defaultdict(<class 'int'>, {'chinese': 1, 'desserts': 1, 'active life': 1, 'restaurants': 2, 'cosmetics & beauty supply': 1, 'fast food': 1, 'taxis': 1, 'pizza': 1, 'professional services': 1, 'automotive': 1, 'pet groomers': 1})}
{'min_lat': 40.08910430705695, 'max_lat': 40.103594960585426, 'min_long': -88.19025298895858, 'max_long': -88.1713489875784, 'category_counts': defaultdict(<class 'int'>, {'bakeries': 1, 'home services': 1, 'plumbing': 1})}
{'min_lat': 40.08910430705695, 'max_lat': 40.103594960585426, 'min_long': -88.1713489875784, 'max_long': -88.15244498619822, 'category_counts': defaultdict(<class 'int'>, {'breweries': 1})}
{'min_lat': 40.103594960585426, 'max_lat': 40.1180856141139, 'min_long': -88.341485, 'max_long': -88.32258099861983, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_l

{'min_lat': 40.1180856141139, 'max_lat': 40.13257626764238, 'min_long': -88.341485, 'max_long': -88.32258099861983, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 40.1180856141139, 'max_lat': 40.13257626764238, 'min_long': -88.32258099861983, 'max_long': -88.30367699723965, 'category_counts': defaultdict(<class 'int'>, {'screen printing': 1})}
{'min_lat': 40.1180856141139, 'max_lat': 40.13257626764238, 'min_long': -88.30367699723965, 'max_long': -88.28477299585947, 'category_counts': defaultdict(<class 'int'>, {'contractors': 1, 'session photography': 1, 'planetarium': 1})}
{'min_lat': 40.1180856141139, 'max_lat': 40.13257626764238, 'min_long': -88.28477299585947, 'max_long': -88.26586899447929, 'category_counts': defaultdict(<class 'int'>, {'nail salons': 1, 'hobby shops': 1, 'professional services': 1, 'local services': 2, 'pizza': 2, 'fast food': 2, 'beauty & spas': 2, 'shopping': 1, 'restaurants': 3, 'wedding planning': 1, 'automot

{'min_lat': 40.147066921170854, 'max_lat': 40.16155757469933, 'min_long': -88.341485, 'max_long': -88.32258099861983, 'category_counts': defaultdict(<class 'int'>, {'no business found in this area': 1})}
{'min_lat': 40.147066921170854, 'max_lat': 40.16155757469933, 'min_long': -88.32258099861983, 'max_long': -88.30367699723965, 'category_counts': defaultdict(<class 'int'>, {'home services': 2})}
{'min_lat': 40.147066921170854, 'max_lat': 40.16155757469933, 'min_long': -88.30367699723965, 'max_long': -88.28477299585947, 'category_counts': defaultdict(<class 'int'>, {'pets': 1})}
{'min_lat': 40.147066921170854, 'max_lat': 40.16155757469933, 'min_long': -88.28477299585947, 'max_long': -88.26586899447929, 'category_counts': defaultdict(<class 'int'>, {'childrens clothing': 1, 'electronics': 1, 'ice cream & frozen yogurt': 1, 'shopping': 1, 'hotels & travel': 1})}
{'min_lat': 40.147066921170854, 'max_lat': 40.16155757469933, 'min_long': -88.26586899447929, 'max_long': -88.24696499309911, 'c

### Write to file

In [12]:
with open('il_grid_boxes_category_counts.json', 'w') as outfile:
    json.dump(boxes, outfile)